# Projeto 9: Regressão carros usados - validação cruzada

## Etapa 1: Importação das bibliotecas

In [2]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import torch.nn as nn        
from skorch import NeuralNetRegressor
import torch
from sklearn.model_selection import cross_val_score
import time
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [4]:
torch.manual_seed(123)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
base = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/autos.csv', encoding = 'ISO-8859-1')

In [7]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [8]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [9]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [10]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values.reshape(-1, 1)

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

In [12]:
previsores = previsores.astype('float32')
preco_real = preco_real.astype('float32')

## Etapa 3: Construção do modelo

In [13]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(316, 158)
        self.dense1 = nn.Linear(158, 158)
        self.dense2 = nn.Linear(158, 1)
        self.activation = nn.ReLU()
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

In [14]:
regressor_sklearn = NeuralNetRegressor(module = regressor_torch, 
                                       criterion = torch.nn.L1Loss, 
                                       optimizer = torch.optim.Adam,
                                       max_epochs = 100,
                                       batch_size = 300,
                                       train_split = False)

## Etapa 4: Validação cruzada

In [18]:
resultados = cross_val_score(regressor_sklearn, previsores, preco_real, cv = 2, #number of iterations
                             scoring = 'neg_mean_absolute_error')

  epoch    train_loss     dur
-------  ------------  ------
      1     3809.9040  4.6431
      2     3221.5167  10.6572
      3     2939.0764  10.6994
      4     2890.9130  6.5764
      5     2869.2301  3.9055
      6     2800.5047  4.0364
      7     2790.3666  5.3665
      8     2758.2663  3.9624
      9     2710.4402  3.9028
     10     2692.9604  5.3888
     11     2688.4882  3.9761
     12     2668.4340  3.9327
     13     2615.1810  5.4258
     14     2589.3750  3.8986
     15     2577.7023  4.1136
     16     2556.1865  5.8998
     17     2585.5145  3.9300
     18     2545.1817  4.0234
     19     2517.5781  5.4802
     20     2505.7310  3.9471
     21     2494.5607  3.9406
     22     2556.8115  5.2071
     23     2743.9237  4.3017
     24     2511.6768  4.2748
     25     2506.4749  5.1467
     26     2471.6389  4.3722
     27     2467.9057  3.8884
     28     2460.7045  4.6586
     29     2463.6477  4.9248
     30     2435.3794  3.9129
     31     2475.5194  4.2541
     32 

In [19]:
media = resultados.mean()
desvio = resultados.std()

In [20]:
print(media)
print(desvio)

-2351.6815185546875
62.8211669921875
